In [145]:
import collections as co
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [167]:
# Webscrape table
URL = "https://www.infoplease.com/world/geography/major-cities-latitude-longitude-and-corresponding-time-zones"
r = requests.get(URL)

soup = BeautifulSoup(r.content, 'html5lib')

table = soup.find('table', attrs = {'id':'A0001770'})
rows = table.find_all('tr')

header = ['City', 'LA1', 'LA2', 'LO1', 'LO2', 'Time'] 
list_of_dicts = []

for row in rows:
    new_row = []
    try:
        for tr in row:
            value = tr.text
            new_row.append(value)
        od = co.OrderedDict(zip(header, new_row))
        list_of_dicts.append(od)

    except AttributeError:
        continue 

# Print example
print(list_of_dicts[2])

OrderedDict([('City', 'Aberdeen, Scotland'), ('LA1', '57'), ('LA2', '9 N'), ('LO1', '2'), ('LO2', '9 W'), ('Time', '5:00 p.m.')])


In [165]:
# Create dataframe
df = pd.DataFrame(list_of_dicts)
df = df.drop([0, 1])
df.head(10)

,City,LA1,LA2,LO1,LO2,Time
2,"Aberdeen, Scotland",57,9 N,2,9 W,5:00 p.m.
3,"Adelaide, Australia",34,55 S,138,36 E,2:30 a.m.1
4,"Algiers, Algeria",36,50 N,3,0 E,6:00 p.m.
5,"Amsterdam, Netherlands",52,22 N,4,53 E,6:00 p.m.
6,"Ankara, Turkey",39,55 N,32,55 E,7:00 p.m.
7,"Asuncin, Paraguay",25,15 S,57,40 W,1:00 p.m.
8,"Athens, Greece",37,58 N,23,43 E,7:00 p.m.
9,"Auckland, New Zealand",36,52 S,174,45 E,5:00 a.m.1
10,"Bangkok, Thailand",13,45 N,100,30 E,midnight
11,"Barcelona, Spain",41,23 N,2,9 E,6:00 p.m.


In [168]:
import math

def degrees_to_radians(degrees):
    return degrees * math.pi / 180

def distance_in_km_between_earth_coordinates(lat1, lon1, lat2, lon2):
    earth_radius_km = 6371

    d_lat = degrees_to_radians(lat2-lat1)
    d_lon = degrees_to_radians(lon2-lon1)

    lat1 = degrees_to_radians(lat1)
    lat2 = degrees_to_radians(lat2)

    a = math.sin(d_lat/2) * math.sin(d_lat/2) + (math.sin(d_lon/2) * math.sin(d_lon/2) * math.cos(lat1) * math.cos(lat2))
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return earth_radius_km * c

# Test
print(distance_in_km_between_earth_coordinates(41.23, 2.09, 13.45, 100.3)) # barcelona bangkok
print(distance_in_km_between_earth_coordinates(28.35, 77.12, 28.35, 77.12))

9696.20047406106
0.0


In [169]:
new_df = pd.DataFrame(columns = ['Start', 'Target', 'Distance'])

# Loop through start locations
for row in df.itertuples():
    row = tuple(row)  # ex. (86, 'New Delhi, India', '28', '35 N', '77', '12 E', '10:30 p.m.')
    
    start_name = row[1].split(",")[0]
    start_lat = float(".".join(row[2:4]).rsplit(" ")[0])  # '28', '35 N' -> 28.35
    start_lon = float(".".join(row[4:6]).rsplit(" ")[0])
    
    # Loop through all target locations
    for target_row in df.itertuples():
        target_row = tuple(target_row)
        
        target_name = target_row[1].split(",")[0]
        target_lat = float(".".join(target_row[2:4]).rsplit(" ")[0])
        target_lon = float(".".join(target_row[4:6]).rsplit(" ")[0])
        
        # Calculate distance between start and target
        distance = distance_in_km_between_earth_coordinates(start_lat, start_lon, target_lat, target_lon)
            
        # Update new dataframe
        new_row = {'Start': start_name, 'Target': target_name, 'Distance': distance}
        new_df = new_df.append(new_row, ignore_index=True)

In [170]:
# Covert datatype
new_df['Distance'] = new_df['Distance'].astype(int)

new_df.head(10)

,Start,Target,Distance
0,Aberdeen,Aberdeen,0
1,Aberdeen,Adelaide,8929
2,Aberdeen,Algiers,2379
3,Aberdeen,Amsterdam,640
4,Aberdeen,Ankara,2938
5,Aberdeen,Asuncin,5588
6,Aberdeen,Athens,2707
7,Aberdeen,Auckland,9486
8,Aberdeen,Bangkok,9173
9,Aberdeen,Barcelona,1853


In [147]:
# Save dataframe as csv file
new_df.to_csv('distances2.csv', index=False)

In [182]:
# Compare result to old csv
df_old = pd.read_csv('distances.csv')
df_old.rename(columns={'Distance':'Distance_old'}, inplace=True)
pd.concat([new_df, df_old.Distance_old], axis=1, join='inner').head(15)

,Start,Target,Distance,Distance_old
0,Aberdeen,Aberdeen,0,0
1,Aberdeen,Adelaide,8929,16192
2,Aberdeen,Algiers,2379,2291
3,Aberdeen,Amsterdam,640,696
4,Aberdeen,Ankara,2938,3158
5,Aberdeen,Asuncin,5588,10521
6,Aberdeen,Athens,2707,2849
7,Aberdeen,Auckland,9486,17748
8,Aberdeen,Bangkok,9173,9469
9,Aberdeen,Barcelona,1853,1779
